In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
from datetime import datetime
import logging
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import queue
import threading


# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('lenta_parser.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


class TatarInformParser:
    def __init__(self, max_workers=5, max_pages=150, timeout=30, delay_range=(1.0, 3.0)):
        self.max_workers = max_workers
        self.max_pages = max_pages
        self.timeout = timeout
        self.delay_range = delay_range
        self.session = self.get_session()
        self.lock = threading.Lock()
        self.processed_pages = 0
        self.total_news = 0
        self.failed_pages = 0


    def get_session(self):
        """Создает сессию с повторными попытками и ротацией User-Agent"""
        session = requests.Session()

        user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1'
        ]

        headers = {
            'User-Agent': random.choice(user_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0',
            'Referer': 'https://lenta.ru/',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin'
        }

        session.headers.update(headers)

        retry_strategy = requests.adapters.Retry(
            total=5,  # Увеличиваем количество попыток
            backoff_factor=1.0,  # Увеличиваем задержку между попытками
            status_forcelist=[429, 500, 502, 503, 504, 403, 404],
        )
        adapter = requests.adapters.HTTPAdapter(max_retries=retry_strategy, pool_connections=20, pool_maxsize=20)
        session.mount("http://", adapter)
        session.mount("https://", adapter)

        return session


    def parse_news_page(self, page_url):
        """Парсит одну страницу с новостями"""
        try:
            delay = random.uniform(*self.delay_range)
            logger.info(f"Задержка {delay:.2f} сек перед запросом к {page_url}")
            time.sleep(delay)

            logger.info(f"Парсим страницу: {page_url}")

            response = self.session.get(page_url, timeout=self.timeout)

            if response.status_code != 200:
                logger.warning(f"Ошибка доступа к {page_url}: {response.status_code}")
                with self.lock:
                    self.failed_pages += 1
                return []

            soup = BeautifulSoup(response.content, 'html.parser')
            news_items = soup.find_all('li', class_='newsList__item')

            # Проверяем, есть ли новости на странице
            if not news_items:
                logger.warning(f"Пустая страница: {page_url}")
                with self.lock:
                    self.failed_pages += 1
                return []

            parsed_news = []
            current_date = datetime.now().strftime("%Y/%m/%d")

            for item in news_items:
                if 'banner' in item.get('class', []) or '_more' in item.get('class', []):
                    continue

                news_link = item.find('a', class_='newsList__item-text')
                if not news_link:
                    continue

                title_elem = news_link.find('h2', class_='newsList__item-title')
                time_elem = news_link.find('a', class_='newsList__item-date')

                if title_elem:
                    href = news_link.get('href', '')
                    if href.startswith('/'):
                        full_url = f'https://tatar-inform.tatar/news{href}'
                    elif href.startswith('http'):
                        full_url = href
                    else:
                        full_url = f'https://tatar-inform.tatar/news{href}'

                    news_data = {
                        'title': title_elem.get_text(strip=True),
                        'url': full_url,
                        'time': time_elem.get_text(strip=True) if time_elem else 'Не указано',
                        'full_date': f"{current_date} {time_elem.get_text(strip=True) if time_elem else ''}",
                        'page_url': page_url,
                        'timestamp': datetime.now().isoformat()
                    }
                    parsed_news.append(news_data)

            with self.lock:
                self.processed_pages += 1
                self.total_news += len(parsed_news)
                logger.info(f"Обработано страниц: {self.processed_pages}, всего новостей: {self.total_news}")

            return parsed_news

        except requests.exceptions.RequestException as e:
            logger.error(f"Сетевая ошибка при парсинге {page_url}: {e}")
            with self.lock:
                self.failed_pages += 1
            return []
        except Exception as e:
            logger.error(f"Ошибка при парсинге {page_url}: {e}")
            with self.lock:
                self.failed_pages += 1
            return []


    def generate_page_urls(self):
        """Генерирует URL всех страниц для парсинга"""
        urls = ["https://tatar-inform.tatar/news"]

        # Генерируем URL для страниц со 2 по max_pages
        for page_num in range(2, self.max_pages + 1):
            urls.append(f"https://tatar-inform.tatar/news?page={page_num}/")

        return urls


    def parse_all_news_pages(self):
        """Парсит все страницы с новостями многопоточно"""
        logger.info(f"Начинаем многопоточный парсинг {self.max_pages} страниц...")

        # Генерируем все URL для парсинга
        urls = self.generate_page_urls()

        logger.info(f"Всего URL для парсинга: {len(urls)}")

        # Многопоточный парсинг с ограничением количества одновременных запросов
        all_news = []

        # Разбиваем на батчи для лучшего управления
        batch_size = self.max_workers * 2
        batches = [urls[i:i + batch_size] for i in range(0, len(urls), batch_size)]

        for batch_num, batch_urls in enumerate(batches, 1):
            logger.info(f"Обрабатываем батч {batch_num}/{len(batches)} ({len(batch_urls)} URL)")

            with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                future_to_url = {executor.submit(self.parse_news_page, url): url for url in batch_urls}

                for future in as_completed(future_to_url):
                    url = future_to_url[future]
                    try:
                        page_news = future.result()
                        if page_news:
                            all_news.extend(page_news)
                            logger.info(f"Успешно: {url} - {len(page_news)} новостей")
                        else:
                            logger.warning(f"Пустая страница или ошибка: {url}")
                    except Exception as e:
                        logger.error(f"Исключение при парсинге {url}: {e}")
                        with self.lock:
                            self.failed_pages += 1

            # Пауза между батчами
            if batch_num < len(batches):
                batch_delay = random.uniform(5.0, 10.0)
                logger.info(f"Пауза между батчами: {batch_delay:.2f} сек")
                time.sleep(batch_delay)

        return all_news


    def parse_news_page_item(self, news):
      """Получение текста по url"""
      page_url = news['url']
      try:
          delay = random.uniform(*self.delay_range)
          logger.info(f"Задержка {delay:.2f} сек перед запросом к {page_url}")
          time.sleep(delay)

          logger.info(f"Парсим страницу: {page_url}")

          response = self.session.get(page_url, timeout=self.timeout)

          if response.status_code != 200:
              logger.warning(f"Ошибка доступа к {page_url}: {response.status_code}")
              with self.lock:
                  self.failed_pages += 1
              return []

          soup = BeautifulSoup(response.content, 'html.parser')
          news_item = soup.find('div', class_='main__news-col')

          # Проверяем, есть ли новость на странице
          if not news_item:
              logger.warning(f"Пустая страница: {page_url}")
              with self.lock:
                  self.failed_pages += 1
              return []

          news_item_title = news_item.find('h1', class_ = 'main__news-title')
          news_item_rubric = news_item.find('a', class_ = 'main__rubric')
          news_item_date = news_item.find('a', class_ = 'main__date')
          news_item_text_div = news_item.find('div', class_ = 'page-main__text')

          news_data = {
              'title': news_item_title.get_text(strip=True),
              'url': page_url,
              'rubric' : news_item_rubric.get_text(strip=True),
              'text' : news_item_text_div.get_text(separator=' ', strip=True),
              'date' : news_item_date.get_text(strip=True)
          }

          return news_data
      except requests.exceptions.RequestException as e:
          logger.error(f"Сетевая ошибка при парсинге {page_url}: {e}")
          with self.lock:
              self.failed_pages += 1
          return {}
      except Exception as e:
          logger.error(f"Ошибка при парсинге {page_url}: {e}")
          with self.lock:
              self.failed_pages += 1
          return {}


    def parse_all_news_page_items(self):
        """Получение текста по всем новостям"""
        all_news = self.parse_all_news_pages()
        full_news = []

        for news in all_news:
          news = self.parse_news_page_item(news)
          full_news.append(news)
          print(news)

        return full_news


    def analyze_results(self, news_list):
        """Анализирует и выводит статистику результатов"""
        if not news_list:
            logger.warning("Нет данных для анализа")
            return

        categories = {}
        pages = {}
        words = 0
        for news in news_list:
            category = news['rubric']
            page = news['url']
            categories[category] = categories.get(category, 0) + 1
            pages[page] = pages.get(page, 0) + 1
            words += len(news['text'].split(' '))

        print("\n" + "="*70)
        print("СТАТИСТИКА МНОГОПОТОЧНОГО ПАРСИНГА TATAR-INFORM.RU")
        print("="*70)
        print(f"Всего новостей: {len(news_list)}")
        print(f"Обработано страниц: {self.processed_pages}")
        print(f"Неудачных страниц: {self.failed_pages}")
        print(f"Всего категорий: {len(categories)}")
        print(f"Количество потоков: {self.max_workers}")
        print(f"Максимальная страница: {self.max_pages}")
        print(f"Число слов: {words}")

        print("\nКоличество новостей по категориям (топ-10):")
        for category, count in sorted(categories.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"  {category}: {count} новостей")

        print(f"\nОбщее количество страниц с новостями: {len(pages)}")


    def convert_to_jsonl(self, data_list, filename):
        """
        Конвертирует список словарей в формат JSONL

        Args:
            data_list: список словарей с данными
            filename: имя файла для сохранения
        """
        with open(filename, 'w', encoding='utf-8') as f:
            for item in data_list:
                json_line = json.dumps(item, ensure_ascii=False)
                f.write(json_line + '\n')


def main():
    """Основная функция"""
    print("🚀 Запуск многопоточного парсера Tatar-Inform.ru до 11 страницы")
    print("="*60)

    try:
        # Создаем парсер с настройками для глубокого парсинга
        parser = TatarInformParser(
            max_workers=8,           # Увеличиваем количество потоков
            max_pages=11,           # Парсим до 11 страницы
            timeout=45,              # Увеличиваем таймаут
            delay_range=(1.5, 4.0)   # Увеличиваем задержку между запросами
        )

        # Парсим все страницы многопоточно
        all_news = parser.parse_all_news_page_items()

        if not all_news:
            print("❌ Не удалось получить новости")
            return

        # Анализируем результаты
        parser.analyze_results(all_news)

        # Сохраняем в jsonl
        jsonl_filename = f"lenta_news_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl"
        parser.convert_to_jsonl(all_news, jsonl_filename)
    except KeyboardInterrupt:
        print("\n⏹️  Парсинг прерван пользователем")
    except Exception as e:
        logger.error(f"Критическая ошибка в main: {e}")
        print(f"❌ Произошла критическая ошибка: {e}")

if __name__ == "__main__":
    main()

🚀 Запуск многопоточного парсера Tatar-Inform.ru до 11 страницы

⏹️  Парсинг прерван пользователем


### Этап 2. Предварительная обработка и очистка текста

In [ ]:
import re
import html
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

class TextCleaner:
    def __init__(self, lowercase=True, remove_stopwords=True, language='russian'):
        """
        Инициализация очистителя текста

        Args:
            lowercase: Приводить текст к нижнему регистру
            remove_stopwords: Удалять стоп-слова
            language: Язык для стоп-слов ('russian', 'tatar', 'english')
        """
        self.lowercase = lowercase
        self.remove_stopwords = remove_stopwords
        self.language = language

        # Загружаем стоп-слова
        if self.remove_stopwords:
            self.stop_words = self._load_stopwords(language)

    def _load_stopwords(self, language):
        """Загрузка стоп-слов для разных языков"""
        if language == 'russian':
            try:
                return set(stopwords.words('russian'))
            except:
                # Резервный список русских стоп-слов
                return self._get_russian_stopwords()

        elif language == 'tatar':
            # Татарские стоп-слова
            return self._get_tatar_stopwords()

        elif language == 'english':
            return set(stopwords.words('english'))

        else:
            print(f"Язык '{language}' не поддерживается. Используем русские стоп-слова.")
            return self._get_russian_stopwords()

    def _get_russian_stopwords(self):
        """Русские стоп-слова"""
        return {
            'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а',
            'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же',
            'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от',
            'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже',
            'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него',
            'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом',
            'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо',
            'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без',
            'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда',
            'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним',
            'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас',
            'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец',
            'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через',
            'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три',
            'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда',
            'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда',
            'конечно', 'всю', 'между'
        }

    def _get_tatar_stopwords(self):
        """Татарские стоп-слова"""
        return {
            'вә', 'һәм', 'белән', 'өчен', 'турында', 'карата', 'буенча', 'аркылы',
            'мин', 'син', 'ул', 'без', 'сез', 'алар', 'монда', 'анда', 'шунда',
            'ни', 'нәрсә', 'кем', 'кайсы', 'ничек', 'никадәр', 'кайчан', 'нишләп',
            'әле', 'инде', 'һаман', 'тагын', 'күбрәк', 'азрак', 'бик', 'бигерәк',
            'тә', 'дә', 'мы', 'ме', 'бы', 'бе', 'гы', 'ге', 'ка', 'кә',
            'ук', 'генә', 'гына', 'чик', 'хәтта', 'әллә', 'гомер', 'көн', 'ел',
            'яки', 'ягъни', 'яисә', 'әгәр', 'чиң', 'шул', 'бу', 'бер', 'ике', 'өч',
            'үз', 'башка', 'барлык', 'бөтен', 'һич', 'бернинди', 'беркайчан',
            'бар', 'юк', 'әйе', 'юк', 'шулай', 'түгел', 'мөмкин', 'кирәк', 'була',
            'дип', 'ди', 'дә', 'тә', 'мыни', 'әйтә', 'күрсәтә', 'алына', 'бирелеп'
        }

    def clean_html(self, text):
        """Удаление HTML-разметки"""
        if not text:
            return ""

        text = html.unescape(text)
        soup = BeautifulSoup(text, 'html.parser')
        clean_text = soup.get_text(separator=' ')
        return clean_text

    def remove_special_characters(self, text):
        """Удаление служебных символов"""
        if not text:
            return ""

        # Удаляем email и URL
        text = re.sub(r'\S*@\S*\s?', '', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'www\S+', '', text)

        # Для татарского сохраняем специфические символы
        if self.language == 'tatar':
            text = re.sub(r'[^\w\s\.\,\!\?\-\:\(\)әөүҗңһ]', '', text)
        else:
            text = re.sub(r'[^\w\s\.\,\!\?\-\:\(\)]', '', text)

        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def normalize_whitespace(self, text):
        """Стандартизация пробельных символов"""
        if not text:
            return ""
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def remove_stopwords_func(self, text):
        """Фильтрация стоп-слов"""
        if not text or not self.remove_stopwords:
            return text

        # Простая токенизация (можно заменить на более сложную)
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in self.stop_words]

        return ' '.join(filtered_words)

    def clean_text(self, text, **kwargs):
        """Основная функция очистки текста"""
        if not text:
            return ""

        cleaned_text = text

        # Применяем очистку HTML
        if kwargs.get('clean_html', True):
            cleaned_text = self.clean_html(cleaned_text)

        # Удаляем специальные символы
        if kwargs.get('remove_special_chars', True):
            cleaned_text = self.remove_special_characters(cleaned_text)

        # Нормализуем пробелы
        if kwargs.get('normalize_whitespace', True):
            cleaned_text = self.normalize_whitespace(cleaned_text)

        # Приводим к нижнему регистру
        if kwargs.get('lowercase', self.lowercase):
            cleaned_text = cleaned_text.lower()

        # Удаляем стоп-слова
        if kwargs.get('remove_stopwords', self.remove_stopwords):
            cleaned_text = self.remove_stopwords_func(cleaned_text)

        return cleaned_text


# Пример использования с татарским языком
if __name__ == "__main__":
    # Татарский текст
    tatar_text = "Мин <p> һәм син бу көнне бик яхшы вакыт үткәрдек<p>. Ул безгә киләчәк."

    cleaner_ru = TextCleaner(language='russian')
    cleaner_tt = TextCleaner(language='tatar')

    print("Татарский текст:")
    print(tatar_text)
    print("\nОчистка как русский текст:")
    print(cleaner_ru.clean_text(tatar_text))
    print("\nОчистка как татарский текст:")
    print(cleaner_tt.clean_text(tatar_text))

Татарский текст:
Мин <p> һәм син бу көнне бик яхшы вакыт үткәрдек<p>. Ул безгә киләчәк.

Очистка как русский текст:
мин һәм син бу көнне бик яхшы вакыт үткәрдек . ул безгә киләчәк.

Очистка как татарский текст:
көнне яхшы вакыт үткәрдек . безгә киләчәк.


In [ ]:
import json

def read_jsonl_basic(filename):
    """Простое чтение JSONL файла"""
    data = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:  # Пропускаем пустые строки
                data.append(json.loads(line))
    return data

# Использование
all_news = read_jsonl_basic('/content/lenta_news_20251006_192459.jsonl')
cleaned_all_news = []

cleaner = TextCleaner(language='tatar')

for news_item in all_news:
    news_item['text_clean'] = cleaner.clean_text(news_item['text'])
    cleaned_all_news.append(news_item)

# for item in cleaned_all_news:
#   print(item)

corpus = [item["text"] for item in cleaned_all_news]
print(corpus)

['Россиядә беренче теплица төзелә башлады, анда банан үстерәчәкләр. Бу хакта Россия авыл хуҗалыгы министры Оксана Лут «Биопром» форумының төп стратегик сессиясендә хәбәр итте . «Бездә хәзер банан үстерү өчен беренче теплица төзелә башлый», – дип министр сүзләрен китерә ТАСС. Узган елның ноябрендә Лут хәбәр иткәнчә, Россия Федерациясендә, Казахстандагы кебек, банан үстерергә планлаштыралар. 2025 елның июлендә Россия Хөкүмәте бананнарны илнең авыл хуҗалыгы продукциясе исемлегенә кертте. Лут сөйләгәнчә, элегрәк министрлыкка төрле үсемлекләр селекциясе белән шөгыльләнүче кеше мөрәҗәгать иткән. Ул Россия Авыл хуҗалыгы министрлыгы хезмәткәрләреннән үзенә безнең илдә үсми торган кызыл киви үсентеләре алырга сорады. Өстәвенә, соңрак министрлык башлыгы ассызыклаганча, объектив рәвештә Россиядә теләсә нәрсә үстерергә мөмкиннәр.', 'Чаллы йогышлы авырулар хастаханәсе табиблары үпкәсе нык зарарланган ир-атны коткарган. Бу хакта медицина учреждениесе матбугат хезмәте хәбәр итә. 51 яшьлек ир-ат хаста

### Этап 3. Проектирование универсального модуля предобработки

In [ ]:
import re
import json
from typing import Dict, List, Any, Optional

class UniversalPreprocessor:
    """
    Универсальный препроцессор для приведения текста к единому стандарту
    """

    def __init__(self, config_path: Optional[str] = None):
        """
        Инициализация препроцессора

        Args:
            config_path: путь к JSON файлу с конфигурацией
        """
        self.default_config = {
            # Стандартизация пунктуации
            'normalize_punctuation': True,
            'normalize_whitespace': True,

            # Замена на токены
            'replace_numbers': True,
            'replace_urls': True,
            'replace_emails': True,
            'replace_currencies': True,
            'replace_phone_numbers': True,

            # Обработка сокращений
            'expand_abbreviations': True,
            'expand_special_abbreviations': True,

            # Дополнительные настройки
            'preserve_sentence_endings': True,
            'remove_extra_spaces': True
        }

        # Загружаем конфигурацию
        self.config = self.default_config.copy()
        if config_path:
            self.load_config(config_path)

        # Инициализируем правила
        self._init_patterns()
        self._init_abbreviations()

    def load_config(self, config_path: str) -> None:
        """Загрузка конфигурации из JSON файла"""
        try:
            with open(config_path, 'r', encoding='utf-8') as f:
                user_config = json.load(f)
            self.config.update(user_config)
        except Exception as e:
            print(f"Ошибка загрузки конфигурации: {e}. Используются настройки по умолчанию.")

    def save_config(self, config_path: str) -> None:
        """Сохранение текущей конфигурации в файл"""
        try:
            with open(config_path, 'w', encoding='utf-8') as f:
                json.dump(self.config, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Ошибка сохранения конфигурации: {e}")

    def _init_patterns(self) -> None:
        """Инициализация регулярных выражений"""

        # Числительные
        self.number_patterns = [
            # Целые числа с разделителями
            (r'\b\d{1,3}(?:[ ,]\d{3})+\b', '<NUM>'),  # 1,000, 10 000
            # Десятичные дроби
            (r'\b\d+[.,]\d+\b', '<NUM>'),  # 3.14, 2,5
            # Простые числа
            (r'\b\d+\b', '<NUM>'),  # 123, 45
        ]

        # URL и email
        self.url_pattern = (r'https?://[^\s]+|www\.[^\s]+', '<URL>')
        self.email_pattern = (r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '<EMAIL>')

        # Номера телефонов
        self.phone_patterns = [
            (r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '<PHONE>'),  # 123-456-7890
            (r'\b\d{1,2}[-.]?\d{3}[-.]?\d{2}[-.]?\d{2}\b', '<PHONE>'),  # 8-912-34-56
            (r'\b\+?[\d\s\-\(\)]{7,}\b', '<PHONE>'),  # Международные форматы
        ]

        # Валюты
        self.currency_pattern = (r'\b\d+[.,]?\d*\s*[₽$€£¥]\b|\b[₽$€£¥]\s*\d+[.,]?\d*\b', '<CURRENCY>')

        # Пунктуация
        self.punctuation_replacements = {
            '…': '...',
            '«': '"',
            '»': '"',
            '„': '"',
            '“': '"',
            '”': '"',
            '‘': "'",
            '’': "'",
            '–': '-',
            '—': '-',
        }

    def _init_abbreviations(self) -> None:
        """Инициализация словаря сокращений"""

        # Общеязыковые сокращения (русский)
        self.common_abbreviations = {
            'т.е.': 'то есть',
            'т.д.': 'так далее',
            'т.п.': 'тому подобное',
            'и т.д.': 'и так далее',
            'и т.п.': 'и тому подобное',
            'и др.': 'и другие',
            'и пр.': 'и прочие',
            'т.к.': 'так как',
            'т.н.': 'так называемый',
            'т.о.': 'таким образом',
            'с.г.': 'сего года',
            'н.э.': 'нашей эры',
            'до н.э.': 'до нашей эры',
            'г.': 'год',
            'гг.': 'годы',
            'вв.': 'века',
            'см.': 'смотри',
            'стр.': 'страница',
            'рис.': 'рисунок',
            'напр.': 'например',
            'мин.': 'минут',
            'сек.': 'секунд',
            'ч.': 'час',
            'кг.': 'килограмм',
            'см.': 'сантиметр',
            'м.': 'метр',
            'км.': 'километр',
            'руб.': 'рубль',
            'долл.': 'доллар',
            'евро.': 'евро',
        }

        # Специальные сокращения (можно расширить)
        self.special_abbreviations = {
            'США': 'Соединенные Штаты Америки',
            'РФ': 'Российская Федерация',
            'СССР': 'Союз Советских Социалистических Республик',
            'ООН': 'Организация Объединенных Наций',
            'НАТО': 'Организация Североатлантического договора',
        }

    def normalize_punctuation(self, text: str) -> str:
        """Стандартизация пунктуации"""
        if not self.config['normalize_punctuation']:
            return text

        for old, new in self.punctuation_replacements.items():
            text = text.replace(old, new)

        return text

    def normalize_whitespace(self, text: str) -> str:
        """Стандартизация пробельных символов"""
        if not self.config['normalize_whitespace']:
            return text

        # Заменяем все пробельные символы на обычные пробелы
        text = re.sub(r'\s+', ' ', text)

        # Убираем пробелы вокруг пунктуации
        text = re.sub(r'\s+([.,!?;:])', r'\1', text)
        text = re.sub(r'([(])\s+', r'\1', text)
        text = re.sub(r'\s+([)])', r'\1', text)

        # Добавляем пробелы после пунктуации, если нужно
        if self.config['preserve_sentence_endings']:
            text = re.sub(r'([.!?])([А-ЯA-Z])', r'\1 \2', text)

        return text.strip()

    def replace_with_tokens(self, text: str) -> str:
        """Замена числительных, URL, email на токены"""

        # URL
        if self.config['replace_urls']:
            pattern, replacement = self.url_pattern
            text = re.sub(pattern, replacement, text)

        # Email
        if self.config['replace_emails']:
            pattern, replacement = self.email_pattern
            text = re.sub(pattern, replacement, text)

        # Числительные
        if self.config['replace_numbers']:
            for pattern, replacement in self.number_patterns:
                text = re.sub(pattern, replacement, text)

        # Номера телефонов
        if self.config['replace_phone_numbers']:
            for pattern, replacement in self.phone_patterns:
                text = re.sub(pattern, replacement, text)

        # Валюты
        if self.config['replace_currencies']:
            pattern, replacement = self.currency_pattern
            text = re.sub(pattern, replacement, text)

        return text

    def expand_abbreviations(self, text: str) -> str:
        """Раскрытие сокращений"""
        if not self.config['expand_abbreviations']:
            return text

        # Общеязыковые сокращения
        for abbrev, expansion in self.common_abbreviations.items():
            # Используем границы слова для точного совпадения
            pattern = r'\b' + re.escape(abbrev) + r'\b'
            text = re.sub(pattern, expansion, text, flags=re.IGNORECASE)

        # Специальные сокращения
        if self.config['expand_special_abbreviations']:
            for abbrev, expansion in self.special_abbreviations.items():
                pattern = r'\b' + re.escape(abbrev) + r'\b'
                text = re.sub(pattern, expansion, text)

        return text

    def preprocess_text(self, text: str, **kwargs) -> str:
        """
        Основная функция предобработки текста

        Args:
            text: исходный текст
            **kwargs: временные настройки (переопределяют конфигурацию)

        Returns:
            обработанный текст
        """
        if not text:
            return ""

        # Сохраняем оригинальную конфигурацию
        original_config = self.config.copy()

        try:
            # Временно применяем переданные настройки
            for key, value in kwargs.items():
                if key in self.config:
                    self.config[key] = value

            # Применяем все этапы обработки
            processed_text = text

            # 1. Стандартизация пунктуации
            processed_text = self.normalize_punctuation(processed_text)

            # 2. Замена на токены
            processed_text = self.replace_with_tokens(processed_text)

            # 3. Раскрытие сокращений
            processed_text = self.expand_abbreviations(processed_text)

            # 4. Стандартизация пробелов
            processed_text = self.normalize_whitespace(processed_text)

            return processed_text

        finally:
            # Восстанавливаем оригинальную конфигурацию
            self.config = original_config

    def batch_preprocess(self, texts: List[str], **kwargs) -> List[str]:
        """Пакетная обработка списка текстов"""
        return [self.preprocess_text(text, **kwargs) for text in texts]


# Функция для быстрой обработки без создания экземпляра
def quick_preprocess(text: str, **kwargs) -> str:
    """
    Быстрая предобработка текста

    Args:
        text: исходный текст
        **kwargs: настройки обработки

    Returns:
        обработанный текст
    """
    processor = UniversalPreprocessor()
    return processor.preprocess_text(text, **kwargs)


# Пример использования и тестирования
if __name__ == "__main__":
    # Тестовые примеры
    test_texts = [
        "Цена: 1,000.50₽. Сайт: https://example.com. Email: test@mail.ru",
        "Текст с сокращениями: т.е., и т.д., см. стр. 5. Номер: 8-912-345-67-89",
        "Разные кавычки «пример» и “пример”. Числа: 3.14, 2,5, 1000000",
        "Организации: США, ООН, НАТО. Дата: 2023 г.",
        "Мин һәм син бу көнне бик яхшы вакыт үткәрдек. Ул безгә киләчәк. 5 тапкыр."
    ]

    # Создаем процессор
    processor = UniversalPreprocessor()

    print("=== Тестирование Universal Preprocessor ===\n")

    for i, text in enumerate(test_texts, 1):
        print(f"Пример {i}:")
        print(f"До: {text}")
        print(f"После: {processor.preprocess_text(text)}")
        print("-" * 50)

    # Демонстрация конфигурации
    print("\n=== Конфигурация ===")
    for key, value in processor.config.items():
        print(f"{key}: {value}")

    # Пример создания конфигурационного файла
    config_example = {
        'normalize_punctuation': True,
        'normalize_whitespace': True,
        'replace_numbers': True,
        'replace_urls': True,
        'replace_emails': True,
        'expand_abbreviations': True,
        'expand_special_abbreviations': False  # Отключаем раскрытие специальных сокращений
    }

    # Сохраняем пример конфигурации
    with open('preprocessor_config.json', 'w', encoding='utf-8') as f:
        json.dump(config_example, f, ensure_ascii=False, indent=2)

    print("\nПример конфигурации сохранен в 'preprocessor_config.json'")

=== Тестирование Universal Preprocessor ===

Пример 1:
До: Цена: 1,000.50₽. Сайт: https://example.com. Email: test@mail.ru
После: Цена: <NUM>.<NUM>₽. Сайт: <URL> Email: <EMAIL>
--------------------------------------------------
Пример 2:
До: Текст с сокращениями: т.е., и т.д., см. стр. 5. Номер: 8-912-345-67-89
После: Текст с сокращениями: т.е., и т.д., см. стр. <NUM>. Номер: <NUM>-<NUM>-<NUM>-<NUM>-<NUM>
--------------------------------------------------
Пример 3:
До: Разные кавычки «пример» и “пример”. Числа: 3.14, 2,5, 1000000
После: Разные кавычки "пример" и "пример". Числа: <NUM>, <NUM>, <NUM>
--------------------------------------------------
Пример 4:
До: Организации: США, ООН, НАТО. Дата: 2023 г.
После: Организации: Соединенные Штаты Америки, Организация Объединенных Наций, Организация Североатлантического договора. Дата: <NUM> г.
--------------------------------------------------
Пример 5:
До: Мин һәм син бу көнне бик яхшы вакыт үткәрдек. Ул безгә киләчәк. 5 тапкыр.
После: Мин

### Этап 4. Сравнительный анализ методов токенизации и нормализации


In [ ]:
preprocessed_data = []

for data in cleaned_all_news:
    data['text'] = quick_preprocess(data['text'])
    preprocessed_data.append(data)

In [ ]:
import pandas as pd
import numpy as np
import time
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import re
import os

# Основные библиотеки для токенизаторов
from tokenizers import Tokenizer
from tokenizers.models import BPE, WordPiece, Unigram
from tokenizers.trainers import BpeTrainer, WordPieceTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers import normalizers

import sentencepiece as spm

class SubwordModelComparator:
    def __init__(self, corpus: List[str]):
        self.corpus = corpus
        self.results = []

    def prepare_corpus_file(self):
        """Сохраняет корпус во временный файл для sentencepiece"""
        with open('temp_corpus.txt', 'w', encoding='utf-8') as f:
            for text in self.corpus:
                if text.strip():  # Пропускаем пустые строки
                    f.write(text.strip() + '\n')
        return 'temp_corpus.txt'

    def calculate_fragmentation(self, tokenized_texts: List[List[str]]) -> float:
        """Вычисляет процент фрагментации слов"""
        total_words = 0
        fragmented_words = 0

        for tokens in tokenized_texts:
            for token in tokens:
                # Определяем фрагментированные токены
                if token.startswith('##') or '▁' in token or len(token) < 3:
                    fragmented_words += 1
                total_words += 1

        return (fragmented_words / total_words * 100) if total_words > 0 else 0

    def calculate_compression_ratio(self, original_texts: List[str], tokenized_texts: List[List[str]]) -> float:
        """Вычисляет коэффициент сжатия"""
        total_original_tokens = sum(len(text.split()) for text in original_texts if text.strip())
        total_subword_tokens = sum(len(tokens) for tokens in tokenized_texts)

        return total_subword_tokens / total_original_tokens if total_original_tokens > 0 else 1

    def reconstruct_text_for_model(self, tokens: List[str], model_name: str) -> str:
        """Правильная реконструкция текста для каждой модели"""
        if not tokens:
            return ""

        if model_name == "Unigram_SP":
            # SentencePiece: '▁' означает начало слова
            text = ''.join(tokens).replace('▁', ' ').strip()

        elif model_name == "WordPiece":
            # WordPiece: '##' означает продолжение слова
            if not tokens:
                return ""
            text = tokens[0]  # Первый токен всегда целый
            for token in tokens[1:]:
                if token.startswith('##'):
                    text += token[2:]  # Убираем '##'
                else:
                    text += ' ' + token

        elif model_name == "BPE":
            # BPE: используем Whitespace, поэтому простой join
            text = ' '.join(tokens)
            # Убираем возможные артефакты маркировки
            text = text.replace(' ##', '').strip()

        elif model_name == "Unigram_HF":
            # Hugging Face Unigram: обычно использует пробелы
            text = ' '.join(tokens).replace(' ##', '').strip()

        else:
            # Fallback
            text = ' '.join(tokens).strip()

        return text

    # def calculate_reconstruction_accuracy(self, original_texts: List[str], reconstructed_texts: List[str]) -> float:
    #     """Вычисляет точность реконструкции с улучшенным сравнением"""
    #     correct = 0
    #     total = min(len(original_texts), len(reconstructed_texts))

    #     for i in range(total):
    #         orig = original_texts[i]
    #         rec = reconstructed_texts[i]

    #         if not orig.strip() or not rec.strip():
    #             continue

    #         # Нормализуем тексты для сравнения
    #         orig_norm = re.sub(r'\s+', ' ', orig.lower().strip())
    #         rec_norm = re.sub(r'\s+', ' ', rec.lower().strip())

    #         # Сравниваем
    #         if orig_norm == rec_norm:
    #             correct += 1
    #         else:
    #             # Для отладки: показываем различия для первых нескольких текстов
    #             if i < 2 and total > 1:  # Показываем только первые 2 различия
    #                 print(f"🔍 Отличие в тексте {i}:")
    #                 print(f"   Оригинал: '{orig[:100]}...'")
    #                 print(f"   Восстановленный: '{rec[:100]}...'")
    #                 print(f"   Совпадение: {orig_norm == rec_norm}")

    #     accuracy = (correct / total * 100) if total > 0 else 0
    #     print(f"   Точность реконструкции: {accuracy:.1f}%")
    #     return accuracy

    def calculate_reconstruction_accuracy(self, original_texts: List[str], reconstructed_texts: List[str]) -> float:
        """Вычисляет точность реконструкции с улучшенной нормализацией пробелов"""
        correct = 0
        total = min(len(original_texts), len(reconstructed_texts))

        for i in range(total):
            orig = original_texts[i]
            rec = reconstructed_texts[i]

            if not orig.strip() or not rec.strip():
                continue

            # УЛУЧШЕННАЯ нормализация: убираем лишние пробелы вокруг знаков препинания
            orig_norm = self.normalize_text(orig)
            rec_norm = self.normalize_text(rec)

            # Сравниваем
            if orig_norm == rec_norm:
                correct += 1
            else:
                # Для отладки: показываем различия для первых нескольких текстов
                if i < 2 and total > 1:
                    print(f"🔍 Отличие в тексте {i}:")
                    print(f"   Оригинал: '{orig_norm[:80]}...'")
                    print(f"   Восстановленный: '{rec_norm[:80]}...'")
                    print(f"   Совпадение: {orig_norm == rec_norm}")

        accuracy = (correct / total * 100) if total > 0 else 0
        print(f"   Точность реконструкции: {accuracy:.1f}% ({correct}/{total})")
        return accuracy

    def normalize_text(self, text: str) -> str:
        """Нормализует текст, убирая лишние пробелы вокруг знаков препинания"""
        # Приводим к нижнему регистру и убираем лишние пробелы
        text = text.lower().strip()

        # Убираем пробелы перед знаками препинания
        text = re.sub(r'\s+([.,!?;:])', r'\1', text)

        # Убираем пробелы после открывающих скобок и перед закрывающими
        text = re.sub(r'\(\s+', '(', text)
        text = re.sub(r'\s+\)', ')', text)

        # Убираем пробелы вокруг дефисов (но сохраняем дефисы в словах)
        text = re.sub(r'\s*-\s*', '-', text)

        # Заменяем multiple spaces на single space
        text = re.sub(r'\s+', ' ', text)

        return text.strip()

    def reconstruct_text_for_model(self, tokens: List[str], model_name: str) -> str:
        """Правильная реконструкция текста для каждой модели"""
        if not tokens:
            return ""

        if model_name == "Unigram_SP":
            # SentencePiece: '▁' означает начало слова
            text = ''.join(tokens).replace('▁', ' ').strip()

        elif model_name == "WordPiece":
            # WordPiece: '##' означает продолжение слова
            if not tokens:
                return ""
            text = tokens[0]
            for token in tokens[1:]:
                if token.startswith('##'):
                    text += token[2:]
                else:
                    text += ' ' + token

        elif model_name == "BPE":
            # BPE: простой join, но убираем лишние пробелы вокруг знаков препинания
            text = ' '.join(tokens)
            # Убираем возможные артефакты маркировки
            text = text.replace(' ##', '')

        elif model_name == "Unigram_HF":
            # Hugging Face Unigram: обычно использует пробелы
            text = ' '.join(tokens).replace(' ##', '')

        else:
            # Fallback
            text = ' '.join(tokens)

        # ПРИМЕНЯЕМ НОРМАЛИЗАЦИЮ К ВОССТАНОВЛЕННОМУ ТЕКСТУ
        return self.normalize_text(text)

    def debug_tokenization(self, model_name: str, tokenized_texts: List[List[str]], num_examples: int = 1):
        """Показывает примеры токенизации для отладки"""
        print(f"   🔍 Пример токенизации ({model_name}):")

        for i in range(min(num_examples, len(tokenized_texts))):
            if i < len(self.corpus):
                original = self.corpus[i]
                tokens = tokenized_texts[i]
                reconstructed = self.reconstruct_text_for_model(tokens, model_name)

                # Показываем только начало текста для краткости
                orig_short = original[:50] + "..." if len(original) > 50 else original
                rec_short = reconstructed[:50] + "..." if len(reconstructed) > 50 else reconstructed

                print(f"     Оригинал: '{orig_short}'")
                print(f"     Токены: {tokens[:10]}{'...' if len(tokens) > 10 else ''}")
                print(f"     Восстановленный: '{rec_short}'")
                print(f"     Совпадение: {original.lower().strip() == reconstructed.lower().strip()}")
                print()

    def train_bpe(self, vocab_size: int, min_frequency: int) -> Tuple[Tokenizer, List[List[str]]]:
        """Обучает BPE модель с Whitespace вместо ByteLevel для кириллицы"""
        tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

        # ИСПРАВЛЕНИЕ: Используем Whitespace вместо ByteLevel для кириллицы
        tokenizer.pre_tokenizer = Whitespace()

        trainer = BpeTrainer(
            vocab_size=vocab_size,
            min_frequency=min_frequency,
            special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"],
            show_progress=False,  # Убираем прогресс-бар для чистоты вывода
        )

        # Обучаем на корпусе
        tokenizer.train_from_iterator(self.corpus, trainer)

        # Токенизируем корпус
        tokenized_texts = []
        for text in self.corpus:
            if text.strip():
                encoding = tokenizer.encode(text)
                tokens = encoding.tokens
                tokenized_texts.append(tokens)

        return tokenizer, tokenized_texts

    def train_wordpiece(self, vocab_size: int, min_frequency: int) -> Tuple[Tokenizer, List[List[str]]]:
        """Обучает WordPiece модель"""
        tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()

        trainer = WordPieceTrainer(
            vocab_size=vocab_size,
            min_frequency=min_frequency,
            special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"],
            show_progress=False,
            continuing_subword_prefix="##"
        )

        # Обучаем на корпусе
        tokenizer.train_from_iterator(self.corpus, trainer)

        # Токенизируем корпус
        tokenized_texts = []
        for text in self.corpus:
            if text.strip():
                encoding = tokenizer.encode(text)
                tokens = encoding.tokens
                tokenized_texts.append(tokens)

        return tokenizer, tokenized_texts

    def train_unigram_sentencepiece(self, vocab_size: int, min_frequency: int) -> Tuple[any, List[List[str]]]:
        """Обучает Unigram модель используя sentencepiece с правильными размерами"""
        corpus_file = self.prepare_corpus_file()

        # Ограничиваем размер словаря для SentencePiece
        max_sentencepiece_vocab = 3000  # Безопасный лимит
        actual_vocab_size = min(vocab_size, max_sentencepiece_vocab)

        model_prefix = f"unigram_model_{actual_vocab_size}"

        try:
            spm.SentencePieceTrainer.train(
                input=corpus_file,
                model_prefix=model_prefix,
                vocab_size=actual_vocab_size,
                model_type='unigram',
                character_coverage=0.9995,
                pad_id=0,
                unk_id=1,
                bos_id=2,
                eos_id=3,
                pad_piece='[PAD]',
                unk_piece='[UNK]',
                bos_piece='[CLS]',
                eos_piece='[SEP]',
                user_defined_symbols=['[MASK]'],
                split_by_whitespace=True,
                max_sentence_length=10000,
            )

            # Загружаем обученную модель
            sp = spm.SentencePieceProcessor()
            sp.load(f"{model_prefix}.model")

            # Токенизируем корпус
            tokenized_texts = []
            for text in self.corpus:
                if text.strip():
                    tokens = sp.encode_as_pieces(text)
                    tokenized_texts.append(tokens)

            return sp, tokenized_texts

        finally:
            # Удаляем временные файлы
            for file in [corpus_file, f"{model_prefix}.model", f"{model_prefix}.vocab"]:
                if os.path.exists(file):
                    try:
                        os.remove(file)
                    except:
                        pass

    def train_unigram_huggingface(self, vocab_size: int, min_frequency: int) -> Tuple[Tokenizer, List[List[str]]]:
        """Обучает Unigram модель через Hugging Face (без ограничений)"""
        tokenizer = Tokenizer(Unigram())
        tokenizer.pre_tokenizer = Whitespace()

        trainer = UnigramTrainer(
            vocab_size=vocab_size,
            special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"],
            unk_token="[UNK]",
            max_piece_length=16,
        )

        # Обучаем на корпусе
        tokenizer.train_from_iterator(self.corpus, trainer)

        # Токенизируем корпус
        tokenized_texts = []
        for text in self.corpus:
            if text.strip():
                encoding = tokenizer.encode(text)
                tokens = encoding.tokens
                tokenized_texts.append(tokens)

        return tokenizer, tokenized_texts

    def analyze_corpus(self):
        """Анализирует корпус для понимания проблемы"""
        print("\n🔍 АНАЛИЗ КОРПУСА:")
        print(f"Общее количество текстов: {len(self.corpus)}")

        # Анализ длины текстов
        text_lengths = [len(text.split()) for text in self.corpus if text.strip()]
        if text_lengths:
            print(f"Средняя длина текста: {np.mean(text_lengths):.1f} слов")

        # Анализ уникальных слов
        all_words = []
        for text in self.corpus:
            if text.strip():
                all_words.extend(text.lower().split())

        unique_words = set(all_words)
        print(f"Уникальных слов в корпусе: {len(unique_words)}")
        print(f"Общее количество слов: {len(all_words)}")

        return len(unique_words)

    def evaluate_model(self, model_name: str, tokenized_texts: List[List[str]],
                      processing_time: float, vocab_size: int) -> Dict:
        """Вычисляет все метрики для модели с исправленной реконструкцией"""

        # Фильтруем пустые тексты
        valid_original = [text for text in self.corpus if text.strip()]
        valid_tokenized = [tokens for tokens in tokenized_texts if tokens]

        if not valid_tokenized:
            return {
                'model': model_name,
                'vocab_size': vocab_size,
                'actual_vocab_size': 0,
                'fragmentation_rate': 0,
                'compression_ratio': 1,
                'reconstruction_accuracy': 0,
                'processing_time_sec': round(processing_time, 2),
                'avg_token_length': 0
            }

        # Процент фрагментации
        fragmentation = self.calculate_fragmentation(valid_tokenized)

        # Коэффициент сжатия
        compression_ratio = self.calculate_compression_ratio(valid_original, valid_tokenized)

        # Реконструкция текста
        reconstructed_texts = []
        for tokens in valid_tokenized:
            reconstructed = self.reconstruct_text_for_model(tokens, model_name)
            reconstructed_texts.append(reconstructed)

        reconstruction_accuracy = self.calculate_reconstruction_accuracy(valid_original, reconstructed_texts)

        # Статистика по токенам
        all_tokens = [token for tokens in valid_tokenized for token in tokens]
        avg_token_length = np.mean([len(token) for token in all_tokens]) if all_tokens else 0
        actual_vocab_size = len(set(all_tokens))

        return {
            'model': model_name,
            'vocab_size': vocab_size,
            'actual_vocab_size': actual_vocab_size,
            'fragmentation_rate': round(fragmentation, 2),
            'compression_ratio': round(compression_ratio, 3),
            'reconstruction_accuracy': round(reconstruction_accuracy, 2),
            'processing_time_sec': round(processing_time, 2),
            'avg_token_length': round(avg_token_length, 2)
        }

    def run_comparison(self, vocab_sizes: List[int] = None, min_frequency: int = 2) -> pd.DataFrame:
        """Запускает сравнительный анализ моделей с правильными размерами словаря"""

        if vocab_sizes is None:
            # Автоматически определяем подходящие размеры
            unique_words = self.analyze_corpus()
            vocab_sizes = [
                min(5000, unique_words),      # Маленький
                min(10000, unique_words * 2), # Средний
                min(20000, unique_words * 3)  # Большой
            ]
            vocab_sizes = [size for size in vocab_sizes if size >= 1000]
            print(f"🎯 Автоматически выбраны размеры словаря: {vocab_sizes}")

        print("🚀 ЗАПУСК СРАВНИТЕЛЬНОГО АНАЛИЗА ПОДСЛОВНЫХ МОДЕЛЕЙ")
        print("=" * 70)

        for vocab_size in vocab_sizes:
            print(f"\n📊 Размер словаря: {vocab_size}")
            print("-" * 50)

            # BPE модель
            print("Обучается BPE...", end=" ")
            start_time = time.time()
            try:
                bpe_model, bpe_tokens = self.train_bpe(vocab_size, min_frequency)
                bpe_time = time.time() - start_time
                self.debug_tokenization("BPE", bpe_tokens)
                bpe_metrics = self.evaluate_model("BPE", bpe_tokens, bpe_time, vocab_size)
                self.results.append(bpe_metrics)
                print("✓")
            except Exception as e:
                print(f"✗ Ошибка: {e}")

            # WordPiece модель
            print("Обучается WordPiece...", end=" ")
            start_time = time.time()
            try:
                wp_model, wp_tokens = self.train_wordpiece(vocab_size, min_frequency)
                wp_time = time.time() - start_time
                self.debug_tokenization("WordPiece", wp_tokens)
                wp_metrics = self.evaluate_model("WordPiece", wp_tokens, wp_time, vocab_size)
                self.results.append(wp_metrics)
                print("✓")
            except Exception as e:
                print(f"✗ Ошибка: {e}")

            # Unigram модель через Hugging Face (без ограничений)
            print("Обучается Unigram (HF)...", end=" ")
            start_time = time.time()
            try:
                unigram_model, unigram_tokens = self.train_unigram_huggingface(vocab_size, min_frequency)
                unigram_time = time.time() - start_time
                self.debug_tokenization("Unigram_HF", unigram_tokens)
                unigram_metrics = self.evaluate_model("Unigram_HF", unigram_tokens, unigram_time, vocab_size)
                self.results.append(unigram_metrics)
                print("✓")
            except Exception as e:
                print(f"✗ Ошибка: {e}")

            # Unigram через SentencePiece (только для маленьких словарей)
            if vocab_size <= 3000:
                print("Обучается Unigram (SP)...", end=" ")
                start_time = time.time()
                try:
                    unigram_sp_model, unigram_sp_tokens = self.train_unigram_sentencepiece(vocab_size, min_frequency)
                    unigram_sp_time = time.time() - start_time
                    self.debug_tokenization("Unigram_SP", unigram_sp_tokens)
                    unigram_sp_metrics = self.evaluate_model("Unigram_SP", unigram_sp_tokens, unigram_sp_time, vocab_size)
                    self.results.append(unigram_sp_metrics)
                    print("✓")
                except Exception as e:
                    print(f"✗ Ошибка: {e}")

        return pd.DataFrame(self.results)

def main():
    """Пример использования"""

    # Ваш корпус (замените на ваши данные)
    corpus = [item["text"] for item in preprocessed_data]

    # Тестовый корпус для примера
    # corpus = [
    #     "Машинное обучение это важный раздел искусственного интеллекта",
    #     "Нейронные сети глубокого обучения используются повсеместно",
    #     "Токенизация текста является фундаментальной задачей NLP",
    #     "Подсловные модели эффективно обрабатывают редкие слова",
    #     "BPE алгоритм широко применяется в предобучении языковых моделей",
    # ] * 66  # 330 текстов

    print(f"📚 Размер корпуса: {len(corpus)} текстов")

    # Создание компаратора
    comparator = SubwordModelComparator(corpus)

    # Запуск сравнения с безопасными размерами словаря
    results_df = comparator.run_comparison(
        vocab_sizes=[2000, 5000, 10000],  # Безопасные размеры
        min_frequency=2
    )

    # Сохранение результатов
    results_df.to_csv('subword_models_comparison.csv', index=False, encoding='utf-8')

    # Вывод результатов
    print("\n" + "="*80)
    print("РЕЗУЛЬТАТЫ СРАВНИТЕЛЬНОГО АНАЛИЗА")
    print("="*80)
    print(results_df.to_string(index=False))

    # Анализ результатов
    if not results_df.empty:
        print("\n" + "="*80)
        print("КЛЮЧЕВЫЕ ВЫВОДЫ")
        print("="*80)

        successful_models = results_df[results_df['actual_vocab_size'] > 100]

        if not successful_models.empty:
            best_fragmentation = successful_models.loc[successful_models['fragmentation_rate'].idxmin()]
            best_compression = successful_models.loc[successful_models['compression_ratio'].idxmin()]
            best_reconstruction = successful_models.loc[successful_models['reconstruction_accuracy'].idxmax()]

            print(f"Лучшая по минимизации фрагментации: {best_fragmentation['model']} ({best_fragmentation['fragmentation_rate']}%)")
            print(f"Лучшая по эффективности сжатия: {best_compression['model']} ({best_compression['compression_ratio']})")
            print(f"Лучшая по точности реконструкции: {best_reconstruction['model']} ({best_reconstruction['reconstruction_accuracy']}%)")

if __name__ == "__main__":
    main()

📚 Размер корпуса: 330 текстов
🚀 ЗАПУСК СРАВНИТЕЛЬНОГО АНАЛИЗА ПОДСЛОВНЫХ МОДЕЛЕЙ

📊 Размер словаря: 2000
--------------------------------------------------
Обучается BPE...    🔍 Пример токенизации (BPE):
     Оригинал: 'Россиядә беренче теплица төзелә башлады, анда бана...'
     Токены: ['Россиядә', 'беренче', 'теплица', 'төзелә', 'баш', 'лады', ',', 'анда', 'банан', 'үстер']...
     Восстановленный: 'россиядә беренче теплица төзелә баш лады, анда бан...'
     Совпадение: False

🔍 Отличие в тексте 0:
   Оригинал: 'россиядә беренче теплица төзелә башлады, анда банан үстерәчәкләр. бу хакта росси...'
   Восстановленный: 'россиядә беренче теплица төзелә баш лады, анда банан үстер әчәк ләр. бу хакта ро...'
   Совпадение: False
🔍 Отличие в тексте 1:
   Оригинал: 'чаллы йогышлы авырулар хастаханәсе табиблары үпкәсе нык зарарланган ир-атны котк...'
   Восстановленный: 'чаллы йо гыш лы авыр улар хастаханә се табиб лары үпкә се нык зарар ланган ир-ат...'
   Совпадение: False
   Точность реконстр

In [ ]:
import pandas as pd
import numpy as np
import time
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import re
import os
import json

# Основные библиотеки для токенизаторов
from tokenizers import Tokenizer
from tokenizers.models import BPE, WordPiece, Unigram
from tokenizers.trainers import BpeTrainer, WordPieceTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers import normalizers

import sentencepiece as spm

class SubwordModelComparator:
    def __init__(self, corpus: List[str]):
        self.corpus = corpus
        self.results = []
        self.trained_models = {}  # 🆕 Храним обученные модели

    def prepare_corpus_file(self):
        """Сохраняет корпус во временный файл для sentencepiece"""
        with open('temp_corpus.txt', 'w', encoding='utf-8') as f:
            for text in self.corpus:
                if text.strip():  # Пропускаем пустые строки
                    f.write(text.strip() + '\n')
        return 'temp_corpus.txt'

    # 🆕 МИНИМАЛЬНОЕ ДОБАВЛЕНИЕ: Методы для работы с обученными моделями
    def save_model(self, model, model_name: str, vocab_size: int):
        """Сохраняет обученную модель в файл"""
        try:
            model_key = f"{model_name}_vocab_{vocab_size}"

            if model_name == "Unigram_SP":
                # SentencePiece модель
                model.save(f"saved_{model_key}.model")
                print(f"✅ Модель {model_key} сохранена")
            else:
                # Hugging Face токенизаторы
                model.save(f"saved_{model_key}")
                print(f"✅ Модель {model_key} сохранена")

            # Сохраняем модель в словарь
            self.trained_models[model_key] = model
            return True

        except Exception as e:
            print(f"❌ Ошибка сохранения {model_name}: {e}")
            return False

    def get_trained_model(self, model_name: str, vocab_size: int):
        """Возвращает обученную модель по имени и размеру словаря"""
        model_key = f"{model_name}_vocab_{vocab_size}"
        return self.trained_models.get(model_key)

    def tokenize_text(self, text: str, model_name: str, vocab_size: int):
        """Токенизирует текст с помощью обученной модели"""
        model = self.get_trained_model(model_name, vocab_size)
        if not model:
            print(f"❌ Модель {model_name} с vocab_size={vocab_size} не найдена")
            return None

        try:
            if model_name == "Unigram_SP":
                return model.encode_as_pieces(text)
            else:
                encoding = model.encode(text)
                return encoding.tokens
        except Exception as e:
            print(f"❌ Ошибка токенизации: {e}")
            return None

    # 🆕 МИНИМАЛЬНОЕ ИЗМЕНЕНИЕ: Добавляем сохранение в методы обучения
    def train_bpe(self, vocab_size: int, min_frequency: int, save_model: bool = True) -> Tuple[Tokenizer, List[List[str]]]:
        """Обучает BPE модель с возможностью сохранения"""
        tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

        # Используем Whitespace вместо ByteLevel для кириллицы
        tokenizer.pre_tokenizer = Whitespace()

        trainer = BpeTrainer(
            vocab_size=vocab_size,
            min_frequency=min_frequency,
            special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"],
            show_progress=False,
        )

        # Обучаем на корпусе
        tokenizer.train_from_iterator(self.corpus, trainer)

        # Токенизируем корпус
        tokenized_texts = []
        for text in self.corpus:
            if text.strip():
                encoding = tokenizer.encode(text)
                tokens = encoding.tokens
                tokenized_texts.append(tokens)

        # 🆕 СОХРАНЯЕМ МОДЕЛЬ
        if save_model:
            self.save_model(tokenizer, "BPE", vocab_size)

        return tokenizer, tokenized_texts

    def train_wordpiece(self, vocab_size: int, min_frequency: int, save_model: bool = True) -> Tuple[Tokenizer, List[List[str]]]:
        """Обучает WordPiece модель с возможностью сохранения"""
        tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()

        trainer = WordPieceTrainer(
            vocab_size=vocab_size,
            min_frequency=min_frequency,
            special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"],
            show_progress=False,
            continuing_subword_prefix="##"
        )

        # Обучаем на корпусе
        tokenizer.train_from_iterator(self.corpus, trainer)

        # Токенизируем корпус
        tokenized_texts = []
        for text in self.corpus:
            if text.strip():
                encoding = tokenizer.encode(text)
                tokens = encoding.tokens
                tokenized_texts.append(tokens)

        # 🆕 СОХРАНЯЕМ МОДЕЛЬ
        if save_model:
            self.save_model(tokenizer, "WordPiece", vocab_size)

        return tokenizer, tokenized_texts

    def train_unigram_sentencepiece(self, vocab_size: int, min_frequency: int, save_model: bool = True) -> Tuple[any, List[List[str]]]:
        """Обучает Unigram модель используя sentencepiece с возможностью сохранения"""
        corpus_file = self.prepare_corpus_file()

        # Ограничиваем размер словаря для SentencePiece
        max_sentencepiece_vocab = 3000
        actual_vocab_size = min(vocab_size, max_sentencepiece_vocab)

        model_prefix = f"unigram_model_{actual_vocab_size}"

        try:
            spm.SentencePieceTrainer.train(
                input=corpus_file,
                model_prefix=model_prefix,
                vocab_size=actual_vocab_size,
                model_type='unigram',
                character_coverage=0.9995,
                pad_id=0,
                unk_id=1,
                bos_id=2,
                eos_id=3,
                pad_piece='[PAD]',
                unk_piece='[UNK]',
                bos_piece='[CLS]',
                eos_piece='[SEP]',
                user_defined_symbols=['[MASK]'],
                split_by_whitespace=True,
                max_sentence_length=10000,
            )

            # Загружаем обученную модель
            sp = spm.SentencePieceProcessor()
            sp.load(f"{model_prefix}.model")

            # Токенизируем корпус
            tokenized_texts = []
            for text in self.corpus:
                if text.strip():
                    tokens = sp.encode_as_pieces(text)
                    tokenized_texts.append(tokens)

            # 🆕 СОХРАНЯЕМ МОДЕЛЬ
            if save_model:
                self.save_model(sp, "Unigram_SP", vocab_size)

            return sp, tokenized_texts

        finally:
            # Удаляем временные файлы
            for file in [corpus_file, f"{model_prefix}.model", f"{model_prefix}.vocab"]:
                if os.path.exists(file):
                    try:
                        os.remove(file)
                    except:
                        pass

    def train_unigram_huggingface(self, vocab_size: int, min_frequency: int, save_model: bool = True) -> Tuple[Tokenizer, List[List[str]]]:
        """Обучает Unigram модель через Hugging Face с возможностью сохранения"""
        tokenizer = Tokenizer(Unigram())
        tokenizer.pre_tokenizer = Whitespace()

        trainer = UnigramTrainer(
            vocab_size=vocab_size,
            special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"],
            unk_token="[UNK]",
            max_piece_length=16,
        )

        # Обучаем на корпусе
        tokenizer.train_from_iterator(self.corpus, trainer)

        # Токенизируем корпус
        tokenized_texts = []
        for text in self.corpus:
            if text.strip():
                encoding = tokenizer.encode(text)
                tokens = encoding.tokens
                tokenized_texts.append(tokens)

        # 🆕 СОХРАНЯЕМ МОДЕЛЬ
        if save_model:
            self.save_model(tokenizer, "Unigram_HF", vocab_size)

        return tokenizer, tokenized_texts

    # 🆕 МИНИМАЛЬНОЕ ДОБАВЛЕНИЕ: Метод для демонстрации использования
    def demonstrate_usage(self, test_text: str = "Пример текста для токенизации"):
        """Демонстрирует использование обученных моделей"""
        print("\n" + "="*60)
        print("🚀 ДЕМОНСТРАЦИЯ ИСПОЛЬЗОВАНИЯ ОБУЧЕННЫХ МОДЕЛЕЙ")
        print("="*60)

        for result in self.results:
            model_name = result['model']
            vocab_size = result['vocab_size']

            tokens = self.tokenize_text(test_text, model_name, vocab_size)
            if tokens:
                print(f"🔹 {model_name} (vocab={vocab_size}):")
                print(f"   Текст: '{test_text}'")
                print(f"   Токены: {tokens}")
                print(f"   Количество токенов: {len(tokens)}")
                print()

    # Все остальные методы остаются без изменений
    def calculate_fragmentation(self, tokenized_texts: List[List[str]]) -> float:
        """Вычисляет процент фрагментации слов"""
        total_words = 0
        fragmented_words = 0

        for tokens in tokenized_texts:
            for token in tokens:
                # Определяем фрагментированные токены
                if token.startswith('##') or '▁' in token or len(token) < 3:
                    fragmented_words += 1
                total_words += 1

        return (fragmented_words / total_words * 100) if total_words > 0 else 0

    def calculate_compression_ratio(self, original_texts: List[str], tokenized_texts: List[List[str]]) -> float:
        """Вычисляет коэффициент сжатия"""
        total_original_tokens = sum(len(text.split()) for text in original_texts if text.strip())
        total_subword_tokens = sum(len(tokens) for tokens in tokenized_texts)

        return total_subword_tokens / total_original_tokens if total_original_tokens > 0 else 1

    def normalize_text(self, text: str) -> str:
        """Нормализует текст, убирая лишние пробелы вокруг знаков препинания"""
        text = text.lower().strip()
        text = re.sub(r'\s+([.,!?;:])', r'\1', text)
        text = re.sub(r'\(\s+', '(', text)
        text = re.sub(r'\s+\)', ')', text)
        text = re.sub(r'\s*-\s*', '-', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def reconstruct_text_for_model(self, tokens: List[str], model_name: str) -> str:
        """Правильная реконструкция текста для каждой модели"""
        if not tokens:
            return ""

        if model_name == "Unigram_SP":
            text = ''.join(tokens).replace('▁', ' ').strip()
        elif model_name == "WordPiece":
            if not tokens:
                return ""
            text = tokens[0]
            for token in tokens[1:]:
                if token.startswith('##'):
                    text += token[2:]
                else:
                    text += ' ' + token
        elif model_name == "BPE":
            text = ' '.join(tokens).replace(' ##', '')
        elif model_name == "Unigram_HF":
            text = ' '.join(tokens).replace(' ##', '')
        else:
            text = ' '.join(tokens)

        return self.normalize_text(text)

    def calculate_reconstruction_accuracy(self, original_texts: List[str], reconstructed_texts: List[str]) -> float:
        """Вычисляет точность реконструкции с улучшенной нормализацией пробелов"""
        correct = 0
        total = min(len(original_texts), len(reconstructed_texts))

        for i in range(total):
            orig = original_texts[i]
            rec = reconstructed_texts[i]

            if not orig.strip() or not rec.strip():
                continue

            orig_norm = self.normalize_text(orig)
            rec_norm = self.normalize_text(rec)

            if orig_norm == rec_norm:
                correct += 1

        accuracy = (correct / total * 100) if total > 0 else 0
        return accuracy

    def debug_tokenization(self, model_name: str, tokenized_texts: List[List[str]], num_examples: int = 1):
        """Показывает примеры токенизации для отладки"""
        print(f"   🔍 Пример токенизации ({model_name}):")

        for i in range(min(num_examples, len(tokenized_texts))):
            if i < len(self.corpus):
                original = self.corpus[i]
                tokens = tokenized_texts[i]
                reconstructed = self.reconstruct_text_for_model(tokens, model_name)

                orig_short = original[:50] + "..." if len(original) > 50 else original
                rec_short = reconstructed[:50] + "..." if len(reconstructed) > 50 else reconstructed

                print(f"     Оригинал: '{orig_short}'")
                print(f"     Токены: {tokens[:10]}{'...' if len(tokens) > 10 else ''}")
                print(f"     Восстановленный: '{rec_short}'")
                print(f"     Совпадение: {original.lower().strip() == reconstructed.lower().strip()}")
                print()

    def analyze_corpus(self):
        """Анализирует корпус для понимания проблемы"""
        print("\n🔍 АНАЛИЗ КОРПУСА:")
        print(f"Общее количество текстов: {len(self.corpus)}")

        text_lengths = [len(text.split()) for text in self.corpus if text.strip()]
        if text_lengths:
            print(f"Средняя длина текста: {np.mean(text_lengths):.1f} слов")

        all_words = []
        for text in self.corpus:
            if text.strip():
                all_words.extend(text.lower().split())

        unique_words = set(all_words)
        print(f"Уникальных слов в корпусе: {len(unique_words)}")
        print(f"Общее количество слов: {len(all_words)}")

        return len(unique_words)

    def evaluate_model(self, model_name: str, tokenized_texts: List[List[str]],
                      processing_time: float, vocab_size: int) -> Dict:
        """Вычисляет все метрики для модели с исправленной реконструкцией"""

        valid_original = [text for text in self.corpus if text.strip()]
        valid_tokenized = [tokens for tokens in tokenized_texts if tokens]

        if not valid_tokenized:
            return {
                'model': model_name,
                'vocab_size': vocab_size,
                'actual_vocab_size': 0,
                'fragmentation_rate': 0,
                'compression_ratio': 1,
                'reconstruction_accuracy': 0,
                'processing_time_sec': round(processing_time, 2),
                'avg_token_length': 0
            }

        fragmentation = self.calculate_fragmentation(valid_tokenized)
        compression_ratio = self.calculate_compression_ratio(valid_original, valid_tokenized)

        reconstructed_texts = []
        for tokens in valid_tokenized:
            reconstructed = self.reconstruct_text_for_model(tokens, model_name)
            reconstructed_texts.append(reconstructed)

        reconstruction_accuracy = self.calculate_reconstruction_accuracy(valid_original, reconstructed_texts)

        all_tokens = [token for tokens in valid_tokenized for token in tokens]
        avg_token_length = np.mean([len(token) for token in all_tokens]) if all_tokens else 0
        actual_vocab_size = len(set(all_tokens))

        return {
            'model': model_name,
            'vocab_size': vocab_size,
            'actual_vocab_size': actual_vocab_size,
            'fragmentation_rate': round(fragmentation, 2),
            'compression_ratio': round(compression_ratio, 3),
            'reconstruction_accuracy': round(reconstruction_accuracy, 2),
            'processing_time_sec': round(processing_time, 2),
            'avg_token_length': round(avg_token_length, 2)
        }

    def run_comparison(self, vocab_sizes: List[int] = None, min_frequency: int = 2) -> pd.DataFrame:
        """Запускает сравнительный анализ моделей с правильными размерами словаря"""

        if vocab_sizes is None:
            unique_words = self.analyze_corpus()
            vocab_sizes = [
                min(5000, unique_words),
                min(10000, unique_words * 2),
                min(20000, unique_words * 3)
            ]
            vocab_sizes = [size for size in vocab_sizes if size >= 1000]
            print(f"🎯 Автоматически выбраны размеры словаря: {vocab_sizes}")

        print("🚀 ЗАПУСК СРАВНИТЕЛЬНОГО АНАЛИЗА ПОДСЛОВНЫХ МОДЕЛЕЙ")
        print("=" * 70)

        for vocab_size in vocab_sizes:
            print(f"\n📊 Размер словаря: {vocab_size}")
            print("-" * 50)

            # BPE модель
            print("Обучается BPE...", end=" ")
            start_time = time.time()
            try:
                bpe_model, bpe_tokens = self.train_bpe(vocab_size, min_frequency)
                bpe_time = time.time() - start_time
                self.debug_tokenization("BPE", bpe_tokens)
                bpe_metrics = self.evaluate_model("BPE", bpe_tokens, bpe_time, vocab_size)
                self.results.append(bpe_metrics)
                print("✓")
            except Exception as e:
                print(f"✗ Ошибка: {e}")

            # WordPiece модель
            print("Обучается WordPiece...", end=" ")
            start_time = time.time()
            try:
                wp_model, wp_tokens = self.train_wordpiece(vocab_size, min_frequency)
                wp_time = time.time() - start_time
                self.debug_tokenization("WordPiece", wp_tokens)
                wp_metrics = self.evaluate_model("WordPiece", wp_tokens, wp_time, vocab_size)
                self.results.append(wp_metrics)
                print("✓")
            except Exception as e:
                print(f"✗ Ошибка: {e}")

            # Unigram модель через Hugging Face
            print("Обучается Unigram (HF)...", end=" ")
            start_time = time.time()
            try:
                unigram_model, unigram_tokens = self.train_unigram_huggingface(vocab_size, min_frequency)
                unigram_time = time.time() - start_time
                self.debug_tokenization("Unigram_HF", unigram_tokens)
                unigram_metrics = self.evaluate_model("Unigram_HF", unigram_tokens, unigram_time, vocab_size)
                self.results.append(unigram_metrics)
                print("✓")
            except Exception as e:
                print(f"✗ Ошибка: {e}")

            # Unigram через SentencePiece
            if vocab_size <= 3000:
                print("Обучается Unigram (SP)...", end=" ")
                start_time = time.time()
                try:
                    unigram_sp_model, unigram_sp_tokens = self.train_unigram_sentencepiece(vocab_size, min_frequency)
                    unigram_sp_time = time.time() - start_time
                    self.debug_tokenization("Unigram_SP", unigram_sp_tokens)
                    unigram_sp_metrics = self.evaluate_model("Unigram_SP", unigram_sp_tokens, unigram_sp_time, vocab_size)
                    self.results.append(unigram_sp_metrics)
                    print("✓")
                except Exception as e:
                    print(f"✗ Ошибка: {e}")

        return pd.DataFrame(self.results)

def main():
    """Пример использования с демонстрацией работы моделей"""

    # Тестовый корпус
    corpus = [
        "Машинное обучение это важный раздел искусственного интеллекта",
        "Нейронные сети глубокого обучения используются повсеместно",
        "Токенизация текста является фундаментальной задачей NLP",
        "Подсловные модели эффективно обрабатывают редкие слова",
        "BPE алгоритм широко применяется в предобучении языковых моделей",
    ] * 20  # 100 текстов

    print(f"📚 Размер корпуса: {len(corpus)} текстов")

    # Создание компаратора
    comparator = SubwordModelComparator(corpus)

    # Запуск сравнения
    results_df = comparator.run_comparison(
        vocab_sizes=[2000, 5000],
        min_frequency=2
    )

    # Сохранение результатов
    results_df.to_csv('subword_models_comparison.csv', index=False, encoding='utf-8')

    # Вывод результатов
    print("\n" + "="*80)
    print("РЕЗУЛЬТАТЫ СРАВНИТЕЛЬНОГО АНАЛИЗА")
    print("="*80)
    print(results_df.to_string(index=False))

    # 🆕 ДЕМОНСТРАЦИЯ ИСПОЛЬЗОВАНИЯ МОДЕЛЕЙ
    test_text = "Машинное обучение и нейронные сети"
    comparator.demonstrate_usage(test_text)

    # 🆕 ПРИМЕР ИСПОЛЬЗОВАНИЯ КОНКРЕТНОЙ МОДЕЛИ
    print("\n" + "="*60)
    print("🎯 ПРИМЕР ИСПОЛЬЗОВАНИЯ КОНКРЕТНОЙ МОДЕЛИ")
    print("="*60)

    # Используем BPE модель с vocab_size=2000
    bpe_tokens = comparator.tokenize_text("Пример текста для обработки", "BPE", 2000)
    if bpe_tokens:
        print(f"BPE модель (vocab=2000): {bpe_tokens}")

    # Анализ результатов
    if not results_df.empty:
        print("\n" + "="*80)
        print("КЛЮЧЕВЫЕ ВЫВОДЫ")
        print("="*80)

        successful_models = results_df[results_df['actual_vocab_size'] > 100]

        if not successful_models.empty:
            best_fragmentation = successful_models.loc[successful_models['fragmentation_rate'].idxmin()]
            best_compression = successful_models.loc[successful_models['compression_ratio'].idxmin()]
            best_reconstruction = successful_models.loc[successful_models['reconstruction_accuracy'].idxmax()]

            print(f"Лучшая по минимизации фрагментации: {best_fragmentation['model']} ({best_fragmentation['fragmentation_rate']}%)")
            print(f"Лучшая по эффективности сжатия: {best_compression['model']} ({best_compression['compression_ratio']})")
            print(f"Лучшая по точности реконструкции: {best_reconstruction['model']} ({best_reconstruction['reconstruction_accuracy']}%)")

if __name__ == "__main__":
    main()

📚 Размер корпуса: 100 текстов
🚀 ЗАПУСК СРАВНИТЕЛЬНОГО АНАЛИЗА ПОДСЛОВНЫХ МОДЕЛЕЙ

📊 Размер словаря: 2000
--------------------------------------------------
Обучается BPE... ✅ Модель BPE_vocab_2000 сохранена
   🔍 Пример токенизации (BPE):
     Оригинал: 'Машинное обучение это важный раздел искусственного...'
     Токены: ['Машинное', 'обучение', 'это', 'важный', 'раздел', 'искусственного', 'интеллекта']
     Восстановленный: 'машинное обучение это важный раздел искусственного...'
     Совпадение: True

✓
Обучается WordPiece... ✅ Модель WordPiece_vocab_2000 сохранена
   🔍 Пример токенизации (WordPiece):
     Оригинал: 'Машинное обучение это важный раздел искусственного...'
     Токены: ['Машинное', 'обучение', 'это', 'важный', 'раздел', 'искусственного', 'интеллекта']
     Восстановленный: 'машинное обучение это важный раздел искусственного...'
     Совпадение: True

✓
Обучается Unigram (HF)... ✅ Модель Unigram_HF_vocab_2000 сохранена
   🔍 Пример токенизации (Unigram_HF):
     Оригинал: 

In [ ]:
# После обучения
corpus = [item["text"] for item in preprocessed_data]
comparator = SubwordModelComparator(corpus)
results_df = comparator.run_comparison(vocab_sizes=[2000, 5000])

# Использование конкретной модели
tokens = comparator.tokenize_text("Мин яратам сине Татарстан", "WordPiece", 2000)
print(f"Токены: {tokens}")

# Или получите саму модель
bpe_model = comparator.get_trained_model("Unigram_HF", 2000)
if bpe_model:
    encoding = bpe_model.encode("Текст")
    tokens = encoding.tokens

🚀 ЗАПУСК СРАВНИТЕЛЬНОГО АНАЛИЗА ПОДСЛОВНЫХ МОДЕЛЕЙ

📊 Размер словаря: 2000
--------------------------------------------------
Обучается BPE... ✅ Модель BPE_vocab_2000 сохранена
   🔍 Пример токенизации (BPE):
     Оригинал: 'Россиядә беренче теплица төзелә башлады, анда бана...'
     Токены: ['Россиядә', 'беренче', 'теплица', 'төзелә', 'баш', 'лады', ',', 'анда', 'банан', 'үстер']...
     Восстановленный: 'россиядә беренче теплица төзелә баш лады, анда бан...'
     Совпадение: False

✓
Обучается WordPiece... ✅ Модель WordPiece_vocab_2000 сохранена
   🔍 Пример токенизации (WordPiece):
     Оригинал: 'Россиядә беренче теплица төзелә башлады, анда бана...'
     Токены: ['Россиядә', 'беренче', 'тепли', '##ца', 'төзелә', 'баш', '##лады', ',', 'анда', 'банан']...
     Восстановленный: 'россиядә беренче теплица төзелә башлады, анда бана...'
     Совпадение: False

✓
Обучается Unigram (HF)... ✅ Модель Unigram_HF_vocab_2000 сохранена
   🔍 Пример токенизации (Unigram_HF):
     Оригинал: 'Россиядә